# Análisis y limpieza de los datos del data set

In [2]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

df_one_star = pd.read_csv('input/one-star-michelin-restaurants.csv')
df_two_stars = pd.read_csv('input/two-stars-michelin-restaurants.csv')
df_three_stars = pd.read_csv('input/three-stars-michelin-restaurants.csv')

In [3]:
print(df_one_star.shape)
print(df_two_stars.shape)
print(df_three_stars.shape)

(549, 10)
(110, 10)
(36, 10)


## Junto los tres data sets

In [4]:
df_one_star["stars"] = 1
df_two_stars["stars"] = 2
df_three_stars["stars"] = 3

In [5]:
df_all = pd.concat([df_one_star, df_two_stars, df_three_stars])
df_all.shape

(695, 11)

## Análisis del data set completo

In [6]:
print(df_all.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 695 entries, 0 to 35
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       695 non-null    object 
 1   year       695 non-null    int64  
 2   latitude   695 non-null    float64
 3   longitude  695 non-null    float64
 4   city       693 non-null    object 
 5   region     695 non-null    object 
 6   zipCode    501 non-null    object 
 7   cuisine    695 non-null    object 
 8   price      519 non-null    object 
 9   url        695 non-null    object 
 10  stars      695 non-null    int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 65.2+ KB
None


In [7]:
# No hay datos duplicados
df_all.duplicated().value_counts()

False    695
dtype: int64

In [8]:
# Algunos restaurantes tienen la ciudad a nulo
df_all[df_all["city"].isnull()]

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,stars
152,Épure,2019,22.295830,114.169304,NaN,Hong Kong,NaN,French,$$$,https://guide.michelin.com/hk/en/hong-kong-reg...,1
166,Arbor,2019,22.283146,114.155420,NaN,Hong Kong,NaN,Innovative,$$$,https://guide.michelin.com/hk/en/hong-kong-reg...,1


In [9]:
# Hay datos de los años 2018 y 2019
df_all["year"].unique()

array([2019, 2018])

In [10]:
# La mayoría de restaurantes son del año 2019
df_all["year"].value_counts()

2019    656
2018     39
Name: year, dtype: int64

In [11]:
# La columna "price" tiene valores nulos
df_all["price"].unique()

array(['$$$$$', '$$$$', '$$$', '$$', '$', nan], dtype=object)

In [12]:
# Los valore nulos en la columna "price" están en las regions "United Kingdom" e "Ireland"
df_all[df_all["price"].isnull()]["region"].value_counts()

United Kingdom    162
Ireland            14
Name: region, dtype: int64

### Restaurantes que aparecen varias veces un mismo año

In [13]:
# Hay algunos restaurantes que aparecen varias veces en un mismo año
df_all.groupby("name").agg({"year": "count"})["year"].sort_values(ascending=False)

name
L'Atelier de Joël Robuchon    3
Sorrel                        2
The Kitchen                   2
Blackbird                     2
Angler                        2
                             ..
Purnell's                     1
Providence                    1
Protégé                       1
Pramerl & the Wolf            1
108                           1
Name: year, Length: 684, dtype: int64

In [14]:
# Este en particular parece que tiene sedes en tres ciudades del mundo y con distinta valoración de estrellas
df_all[df_all["name"] == "L'Atelier de Joël Robuchon"]

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,stars
360,L'Atelier de Joël Robuchon,2019,25.039188,121.56771,Taipei,Taipei,110,French contemporary,$$$,https://guide.michelin.com/tw/en/taipei-region...,1
47,L'Atelier de Joël Robuchon,2019,40.742905,-74.00769,New York,New York City,NaN,French,$$$$,https://guide.michelin.com/us/en/new-york-stat...,2
15,L'Atelier de Joël Robuchon,2019,22.281199,114.15816,Hong Kong,Hong Kong,NaN,French contemporary,$$$$,https://guide.michelin.com/hk/en/hong-kong-reg...,3


In [15]:
# Puede que sean restaurantes diferentes pero a la hora de seleccionar por nombre aparecerán dos restaurantes
df_all[df_all["name"] == "The Kitchen"]

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,stars
57,The Kitchen,2019,38.588940,-121.41424,Sacramento,California,95825,Contemporary,$$$$,https://guide.michelin.com/us/en/california/us...,1
186,The Kitchen,2019,22.191442,113.54300,Macau,Macau,NaN,Steakhouse,$$$,https://guide.michelin.com/mo/en/macau-region/...,1


### Restaurantes que aparecen en 2019 y 2018

In [16]:
# Hay un restaurante que aparece en 2018 y en 2019
df_all[df_all["year"] == 2018][["name"]].merge(df_all[df_all["year"] == 2019][["name"]])

,name
0,Summer Palace


In [17]:
# Parece que tiene dos sedes en ciudades diferentes
df_all[df_all["name"] == "Summer Palace"]

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,stars
164,Summer Palace,2019,22.277136,114.1643,Hong Kong,Hong Kong,NaN,Cantonese,$$$,https://guide.michelin.com/hk/en/hong-kong-reg...,1
333,Summer Palace,2018,1.304385,103.8250,Singapore,Singapore,NaN,Cantonese,$,https://guide.michelin.com/sg/en/singapore-reg...,1


# Hora de juntar datos del dataframe con los de la api
Los campos por los que juntar son:
- latitude
- longitude

In [18]:
# Voy a probar con datos de este último restaurante:
df_test = df_all[df_all["name"] == "Summer Palace"]

In [19]:
load_dotenv()
key = os.getenv("OPENWEATHERMAP_APIKEY")

In [22]:
units = "metric" # Temperatura en Celsius

list_of_restaurants = []

for name, city, region, cuisine, price, url, stars, latitude, longitude in df_test[["name", "city", "region", "cuisine", "price", "url", "stars", "latitude", "longitude"]].itertuples(index=False):
    url = f'http://api.openweathermap.org/data/2.5/onecall?lat={latitude}&lon={longitude}&units={units}&APPID={key}'

    response = requests.get(url)
    
    data = response.json()
    
    restaurant = {'name': name,
                 'city': city,
                 'region': region,
                 'cuisine': cuisine,
                 'price': price,
                 'url': url,
                 'stars': stars,
                 'current_temp': data['current']['temp'],
                 'current_feels_like': data['current']['feels_like']
                 }

    try:
        if data['current']['rain']:
            restaurant['rain_1h'] = data['current']['rain']['1h']
    except KeyError:
        restaurant['rain_1h'] = "En esta localización no hay información sobre previsión de lluvia."

    try:
        if data['current']['weather']:
            restaurant['weather'] = data['current']['weather'][0]['main']
            restaurant['weather_description'] = data['current']['weather'][0]['description']
    except KeyError:
        restaurant['weather'] = "En esta localización no hay información meteorológica."
        restaurant['weather_description'] = "En esta localización no hay información meteorológica."

    list_of_restaurants.append(restaurant)

In [21]:
for restaurant in list_of_restaurants:
    print(restaurant)

{'name': 'Summer Palace', 'city': 'Hong Kong', 'region': 'Hong Kong', 'cuisine': 'Cantonese', 'price': '$$$', 'url': 'http://api.openweathermap.org/data/2.5/onecall?lat=22.277136&lon=114.1643&units=metric&APPID=0964509b18e74504c35ad703745a7e91', 'stars': 1, 'current_temp': 28.17, 'current_feels_like': 31.78, 'rain_1h': 'En esta localización no hay información sobre previsión de lluvia.', 'weather': 'Clouds', 'weather_description': 'few clouds'}
{'name': 'Summer Palace', 'city': 'Singapore', 'region': 'Singapore', 'cuisine': 'Cantonese', 'price': '$', 'url': 'http://api.openweathermap.org/data/2.5/onecall?lat=1.3043848&lon=103.825&units=metric&APPID=0964509b18e74504c35ad703745a7e91', 'stars': 1, 'current_temp': 26.83, 'current_feels_like': 31.42, 'rain_1h': 1.89, 'weather': 'Rain', 'weather_description': 'moderate rain'}


### APÉNDICE:
Ejemplos de JSON devueltos por la API.

<b>Ejemplo de JSON devuelto 1</b>:

```
{'lat': 22.28, 'lon': 114.16, 'timezone': 'Asia/Hong_Kong', 'timezone_offset': 28800, 'current': {'dt': 1600080717, 'sunrise': 1600034997, 'sunset': 1600079290, 'temp': 28.17, 'feels_like': 31.78, 'pressure': 1008, 'humidity': 83, 'dew_point': 25.01, 'uvi': 11.88, 'clouds': 13, 'visibility': 10000, 'wind_speed': 4.02, 'wind_deg': 90, 'wind_gust': 6.71, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}]}, 'minutely': [{'dt': 1600080720, 'precipitation': 0}, {'dt': 1600080780, 'precipitation': 0}, {'dt': 1600080840, 'precipitation': 0}, {'dt': 1600080900, 'precipitation': 0}, {'dt': 1600080960, 'precipitation': 0}, {'dt': 1600081020, 'precipitation': 0}, {'dt': 1600081080, 'precipitation': 0}, {'dt': 1600081140, 'precipitation': 0}, {'dt': 1600081200, 'precipitation': 0}, {'dt': 1600081260, 'precipitation': 0}, {'dt': 1600081320, 'precipitation': 0}, {'dt': 1600081380, 'precipitation': 0}, {'dt': 1600081440, 'precipitation': 0}, {'dt': 1600081500, 'precipitation': 0}, {'dt': 1600081560, 'precipitation': 0}, {'dt': 1600081620, 'precipitation': 0}, {'dt': 1600081680, 'precipitation': 0}, {'dt': 1600081740, 'precipitation': 0}, {'dt': 1600081800, 'precipitation': 0}, {'dt': 1600081860, 'precipitation': 0}, {'dt': 1600081920, 'precipitation': 0}, {'dt': 1600081980, 'precipitation': 0}, {'dt': 1600082040, 'precipitation': 0}, {'dt': 1600082100, 'precipitation': 0}, {'dt': 1600082160, 'precipitation': 0}, {'dt': 1600082220, 'precipitation': 0}, {'dt': 1600082280, 'precipitation': 0}, {'dt': 1600082340, 'precipitation': 0}, {'dt': 1600082400, 'precipitation': 0}, {'dt': 1600082460, 'precipitation': 0}, {'dt': 1600082520, 'precipitation': 0}, {'dt': 1600082580, 'precipitation': 0}, {'dt': 1600082640, 'precipitation': 0}, {'dt': 1600082700, 'precipitation': 0}, {'dt': 1600082760, 'precipitation': 0}, {'dt': 1600082820, 'precipitation': 0}, {'dt': 1600082880, 'precipitation': 0}, {'dt': 1600082940, 'precipitation': 0}, {'dt': 1600083000, 'precipitation': 0}, {'dt': 1600083060, 'precipitation': 0}, {'dt': 1600083120, 'precipitation': 0}, {'dt': 1600083180, 'precipitation': 0}, {'dt': 1600083240, 'precipitation': 0}, {'dt': 1600083300, 'precipitation': 0}, {'dt': 1600083360, 'precipitation': 0}, {'dt': 1600083420, 'precipitation': 0}, {'dt': 1600083480, 'precipitation': 0}, {'dt': 1600083540, 'precipitation': 0}, {'dt': 1600083600, 'precipitation': 0}, {'dt': 1600083660, 'precipitation': 0}, {'dt': 1600083720, 'precipitation': 0}, {'dt': 1600083780, 'precipitation': 0}, {'dt': 1600083840, 'precipitation': 0}, {'dt': 1600083900, 'precipitation': 0}, {'dt': 1600083960, 'precipitation': 0}, {'dt': 1600084020, 'precipitation': 0}, {'dt': 1600084080, 'precipitation': 0}, {'dt': 1600084140, 'precipitation': 0}, {'dt': 1600084200, 'precipitation': 0}, {'dt': 1600084260, 'precipitation': 0}, {'dt': 1600084320, 'precipitation': 0}], 'hourly': [{'dt': 1600077600, 'temp': 28.17, 'feels_like': 28.96, 'pressure': 1008, 'humidity': 83, 'dew_point': 25.01, 'clouds': 13, 'visibility': 10000, 'wind_speed': 8.04, 'wind_deg': 97, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'pop': 0.73}, {'dt': 1600081200, 'temp': 28.36, 'feels_like': 28.86, 'pressure': 1008, 'humidity': 81, 'dew_point': 24.78, 'clouds': 12, 'visibility': 10000, 'wind_speed': 8.26, 'wind_deg': 96, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'pop': 0.73}, {'dt': 1600084800, 'temp': 28.49, 'feels_like': 29.03, 'pressure': 1009, 'humidity': 80, 'dew_point': 24.7, 'clouds': 16, 'visibility': 10000, 'wind_speed': 8.13, 'wind_deg': 92, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'pop': 0.78}, {'dt': 1600088400, 'temp': 28.59, 'feels_like': 29.3, 'pressure': 1009, 'humidity': 80, 'dew_point': 24.8, 'clouds': 84, 'visibility': 10000, 'wind_speed': 7.97, 'wind_deg': 90, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'pop': 0.67}, {'dt': 1600092000, 'temp': 28.59, 'feels_like': 29.14, 'pressure': 1010, 'humidity': 79, 'dew_point': 24.59, 'clouds': 93, 'visibility': 10000, 'wind_speed': 8.02, 'wind_deg': 92, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.63, 'rain': {'1h': 0.54}}, {'dt': 1600095600, 'temp': 28.63, 'feels_like': 29.05, 'pressure': 1010, 'humidity': 79, 'dew_point': 24.69, 'clouds': 96, 'visibility': 10000, 'wind_speed': 8.24, 'wind_deg': 92, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.67, 'rain': {'1h': 0.75}}, {'dt': 1600099200, 'temp': 28.46, 'feels_like': 28.81, 'pressure': 1010, 'humidity': 79, 'dew_point': 24.65, 'clouds': 97, 'visibility': 10000, 'wind_speed': 8.2, 'wind_deg': 91, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10n'}], 'pop': 0.73, 'rain': {'1h': 1.14}}, {'dt': 1600102800, 'temp': 28.36, 'feels_like': 28.54, 'pressure': 1010, 'humidity': 80, 'dew_point': 24.63, 'clouds': 98, 'visibility': 10000, 'wind_speed': 8.54, 'wind_deg': 88, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.73, 'rain': {'1h': 0.83}}, {'dt': 1600106400, 'temp': 28.16, 'feels_like': 28.1, 'pressure': 1009, 'humidity': 81, 'dew_point': 24.63, 'clouds': 98, 'visibility': 10000, 'wind_speed': 8.9, 'wind_deg': 88, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.75, 'rain': {'1h': 0.43}}, {'dt': 1600110000, 'temp': 27.88, 'feels_like': 27.91, 'pressure': 1009, 'humidity': 82, 'dew_point': 24.63, 'clouds': 100, 'visibility': 10000, 'wind_speed': 8.7, 'wind_deg': 86, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.83, 'rain': {'1h': 0.68}}, {'dt': 1600113600, 'temp': 27.81, 'feels_like': 27.84, 'pressure': 1009, 'humidity': 82, 'dew_point': 24.63, 'clouds': 100, 'visibility': 10000, 'wind_speed': 8.65, 'wind_deg': 86, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.83, 'rain': {'1h': 0.75}}, {'dt': 1600117200, 'temp': 27.78, 'feels_like': 27.98, 'pressure': 1009, 'humidity': 83, 'dew_point': 24.66, 'clouds': 100, 'visibility': 10000, 'wind_speed': 8.55, 'wind_deg': 87, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.84, 'rain': {'1h': 0.46}}, {'dt': 1600120800, 'temp': 27.71, 'feels_like': 28.08, 'pressure': 1009, 'humidity': 83, 'dew_point': 24.65, 'clouds': 100, 'visibility': 10000, 'wind_speed': 8.25, 'wind_deg': 87, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.84, 'rain': {'1h': 0.4}}, {'dt': 1600124400, 'temp': 27.69, 'feels_like': 28.37, 'pressure': 1010, 'humidity': 83, 'dew_point': 24.57, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.79, 'wind_deg': 87, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.84, 'rain': {'1h': 0.56}}, {'dt': 1600128000, 'temp': 27.81, 'feels_like': 28.76, 'pressure': 1010, 'humidity': 82, 'dew_point': 24.54, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.34, 'wind_deg': 86, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.88, 'rain': {'1h': 0.85}}, {'dt': 1600131600, 'temp': 28.02, 'feels_like': 29.12, 'pressure': 1010, 'humidity': 81, 'dew_point': 24.62, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.12, 'wind_deg': 86, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.96, 'rain': {'1h': 0.82}}, {'dt': 1600135200, 'temp': 28.26, 'feels_like': 29.55, 'pressure': 1010, 'humidity': 81, 'dew_point': 24.7, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.05, 'wind_deg': 84, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.92, 'rain': {'1h': 0.43}}, {'dt': 1600138800, 'temp': 28.45, 'feels_like': 29.62, 'pressure': 1010, 'humidity': 80, 'dew_point': 24.77, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.2, 'wind_deg': 86, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.97, 'rain': {'1h': 0.25}}, {'dt': 1600142400, 'temp': 28.64, 'feels_like': 29.99, 'pressure': 1009, 'humidity': 80, 'dew_point': 24.89, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.1, 'wind_deg': 87, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.99, 'rain': {'1h': 0.22}}, {'dt': 1600146000, 'temp': 28.79, 'feels_like': 30.3, 'pressure': 1009, 'humidity': 79, 'dew_point': 24.89, 'clouds': 100, 'visibility': 10000, 'wind_speed': 6.82, 'wind_deg': 94, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.99, 'rain': {'1h': 0.34}}, {'dt': 1600149600, 'temp': 28.94, 'feels_like': 30.5, 'pressure': 1008, 'humidity': 78, 'dew_point': 24.86, 'clouds': 100, 'visibility': 10000, 'wind_speed': 6.69, 'wind_deg': 101, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.99, 'rain': {'1h': 0.16}}, {'dt': 1600153200, 'temp': 28.97, 'feels_like': 30.76, 'pressure': 1007, 'humidity': 78, 'dew_point': 24.93, 'clouds': 99, 'visibility': 10000, 'wind_speed': 6.39, 'wind_deg': 105, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.36}, {'dt': 1600156800, 'temp': 28.86, 'feels_like': 30.88, 'pressure': 1007, 'humidity': 80, 'dew_point': 25.11, 'clouds': 99, 'visibility': 10000, 'wind_speed': 6.34, 'wind_deg': 106, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.33}, {'dt': 1600160400, 'temp': 28.55, 'feels_like': 30.71, 'pressure': 1007, 'humidity': 82, 'dew_point': 25.31, 'clouds': 96, 'visibility': 10000, 'wind_speed': 6.24, 'wind_deg': 103, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.29}, {'dt': 1600164000, 'temp': 28.37, 'feels_like': 30.62, 'pressure': 1007, 'humidity': 84, 'dew_point': 25.47, 'clouds': 96, 'visibility': 10000, 'wind_speed': 6.32, 'wind_deg': 99, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.2}, {'dt': 1600167600, 'temp': 28.26, 'feels_like': 30.38, 'pressure': 1007, 'humidity': 85, 'dew_point': 25.59, 'clouds': 96, 'visibility': 10000, 'wind_speed': 6.59, 'wind_deg': 97, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0.23}, {'dt': 1600171200, 'temp': 28.23, 'feels_like': 30.36, 'pressure': 1008, 'humidity': 86, 'dew_point': 25.73, 'clouds': 97, 'visibility': 10000, 'wind_speed': 6.72, 'wind_deg': 97, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0.21}, {'dt': 1600174800, 'temp': 28.23, 'feels_like': 30.15, 'pressure': 1009, 'humidity': 86, 'dew_point': 25.73, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.02, 'wind_deg': 98, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.45, 'rain': {'1h': 0.14}}, {'dt': 1600178400, 'temp': 28.25, 'feels_like': 29.94, 'pressure': 1009, 'humidity': 85, 'dew_point': 25.68, 'clouds': 100, 'visibility': 10000, 'wind_speed': 7.19, 'wind_deg': 99, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.57, 'rain': {'1h': 0.2}}, {'dt': 1600182000, 'temp': 28.3, 'feels_like': 30.18, 'pressure': 1009, 'humidity': 85, 'dew_point': 25.6, 'clouds': 95, 'visibility': 10000, 'wind_speed': 6.96, 'wind_deg': 101, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.71, 'rain': {'1h': 0.61}}, {'dt': 1600185600, 'temp': 28.24, 'feels_like': 30.29, 'pressure': 1009, 'humidity': 85, 'dew_point': 25.52, 'clouds': 88, 'visibility': 10000, 'wind_speed': 6.67, 'wind_deg': 105, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.81, 'rain': {'1h': 0.63}}, {'dt': 1600189200, 'temp': 28.12, 'feels_like': 30.33, 'pressure': 1009, 'humidity': 84, 'dew_point': 25.33, 'clouds': 87, 'visibility': 10000, 'wind_speed': 6.15, 'wind_deg': 109, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.85, 'rain': {'1h': 0.86}}, {'dt': 1600192800, 'temp': 28.01, 'feels_like': 30.46, 'pressure': 1008, 'humidity': 84, 'dew_point': 25.13, 'clouds': 86, 'visibility': 10000, 'wind_speed': 5.71, 'wind_deg': 115, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10n'}], 'pop': 0.93, 'rain': {'1h': 1.11}}, {'dt': 1600196400, 'temp': 28.01, 'feels_like': 30.87, 'pressure': 1008, 'humidity': 84, 'dew_point': 25.11, 'clouds': 80, 'visibility': 10000, 'wind_speed': 5.13, 'wind_deg': 114, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.89, 'rain': {'1h': 0.83}}, {'dt': 1600200000, 'temp': 28.03, 'feels_like': 31.04, 'pressure': 1008, 'humidity': 83, 'dew_point': 25.06, 'clouds': 79, 'visibility': 10000, 'wind_speed': 4.76, 'wind_deg': 115, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.89, 'rain': {'1h': 0.65}}, {'dt': 1600203600, 'temp': 28.08, 'feels_like': 31.23, 'pressure': 1008, 'humidity': 83, 'dew_point': 25.04, 'clouds': 77, 'visibility': 10000, 'wind_speed': 4.6, 'wind_deg': 115, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.89, 'rain': {'1h': 0.47}}, {'dt': 1600207200, 'temp': 28.14, 'feels_like': 31.5, 'pressure': 1008, 'humidity': 83, 'dew_point': 25.16, 'clouds': 75, 'visibility': 10000, 'wind_speed': 4.35, 'wind_deg': 116, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.89, 'rain': {'1h': 0.29}}, {'dt': 1600210800, 'temp': 28.38, 'feels_like': 31.89, 'pressure': 1009, 'humidity': 83, 'dew_point': 25.33, 'clouds': 69, 'visibility': 10000, 'wind_speed': 4.34, 'wind_deg': 114, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.91, 'rain': {'1h': 0.35}}, {'dt': 1600214400, 'temp': 28.71, 'feels_like': 32.35, 'pressure': 1009, 'humidity': 82, 'dew_point': 25.39, 'clouds': 60, 'visibility': 10000, 'wind_speed': 4.27, 'wind_deg': 110, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.92, 'rain': {'1h': 0.16}}, {'dt': 1600218000, 'temp': 28.99, 'feels_like': 32.46, 'pressure': 1010, 'humidity': 81, 'dew_point': 25.5, 'clouds': 34, 'visibility': 10000, 'wind_speed': 4.57, 'wind_deg': 103, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'pop': 0.6}, {'dt': 1600221600, 'temp': 29.13, 'feels_like': 32.17, 'pressure': 1010, 'humidity': 80, 'dew_point': 25.47, 'clouds': 66, 'visibility': 10000, 'wind_speed': 5.12, 'wind_deg': 97, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'pop': 0.53}, {'dt': 1600225200, 'temp': 29.16, 'feels_like': 31.7, 'pressure': 1009, 'humidity': 80, 'dew_point': 25.41, 'clouds': 77, 'visibility': 10000, 'wind_speed': 5.86, 'wind_deg': 91, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'pop': 0.5}, {'dt': 1600228800, 'temp': 29.16, 'feels_like': 31.44, 'pressure': 1009, 'humidity': 79, 'dew_point': 25.32, 'clouds': 74, 'visibility': 10000, 'wind_speed': 6.04, 'wind_deg': 89, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'pop': 0.43}, {'dt': 1600232400, 'temp': 29.21, 'feels_like': 31.41, 'pressure': 1008, 'humidity': 79, 'dew_point': 25.32, 'clouds': 62, 'visibility': 10000, 'wind_speed': 6.2, 'wind_deg': 92, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'pop': 0.38}, {'dt': 1600236000, 'temp': 29.5, 'feels_like': 31.82, 'pressure': 1008, 'humidity': 79, 'dew_point': 25.56, 'clouds': 52, 'visibility': 10000, 'wind_speed': 6.27, 'wind_deg': 96, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'pop': 0.36}, {'dt': 1600239600, 'temp': 29.59, 'feels_like': 31.94, 'pressure': 1007, 'humidity': 78, 'dew_point': 25.52, 'clouds': 1, 'visibility': 10000, 'wind_speed': 6.11, 'wind_deg': 108, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'pop': 0}, {'dt': 1600243200, 'temp': 29.59, 'feels_like': 32.13, 'pressure': 1006, 'humidity': 77, 'dew_point': 25.22, 'clouds': 6, 'visibility': 10000, 'wind_speed': 5.65, 'wind_deg': 106, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'pop': 0}, {'dt': 1600246800, 'temp': 29.35, 'feels_like': 31.63, 'pressure': 1006, 'humidity': 78, 'dew_point': 25.31, 'clouds': 4, 'visibility': 10000, 'wind_speed': 6.01, 'wind_deg': 101, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'pop': 0}], 'daily': [{'dt': 1600056000, 'sunrise': 1600034997, 'sunset': 1600079290, 'temp': {'day': 28.55, 'min': 27.14, 'max': 29, 'night': 28.53, 'eve': 28.17, 'morn': 27.14}, 'feels_like': {'day': 29.39, 'night': 28.89, 'eve': 28.99, 'morn': 28.08}, 'pressure': 1010, 'humidity': 78, 'dew_point': 24.49, 'wind_speed': 7.39, 'wind_deg': 77, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10d'}], 'clouds': 55, 'pop': 0.81, 'rain': 19.18, 'uvi': 11.88}, {'dt': 1600142400, 'sunrise': 1600121413, 'sunset': 1600165630, 'temp': {'day': 28.45, 'min': 27.78, 'max': 28.94, 'night': 28.3, 'eve': 28.55, 'morn': 27.78}, 'feels_like': {'day': 29.62, 'night': 30.18, 'eve': 30.71, 'morn': 27.98}, 'pressure': 1010, 'humidity': 80, 'dew_point': 24.77, 'wind_speed': 7.2, 'wind_deg': 86, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 100, 'pop': 0.99, 'rain': 9.37, 'uvi': 11.83}, {'dt': 1600228800, 'sunrise': 1600207829, 'sunset': 1600251970, 'temp': {'day': 29.16, 'min': 28.01, 'max': 29.5, 'night': 28.77, 'eve': 29.35, 'morn': 28.08}, 'feels_like': {'day': 31.7, 'night': 29.82, 'eve': 31.63, 'morn': 31.23}, 'pressure': 1009, 'humidity': 80, 'dew_point': 25.41, 'wind_speed': 5.86, 'wind_deg': 91, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 77, 'pop': 0.93, 'rain': 5.33, 'uvi': 12.67}, {'dt': 1600315200, 'sunrise': 1600294245, 'sunset': 1600338309, 'temp': {'day': 29.29, 'min': 28.29, 'max': 29.69, 'night': 28.44, 'eve': 29.18, 'morn': 28.29}, 'feels_like': {'day': 29.58, 'night': 27.2, 'eve': 28.51, 'morn': 29.39}, 'pressure': 1008, 'humidity': 79, 'dew_point': 25.34, 'wind_speed': 8.99, 'wind_deg': 74, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 100, 'pop': 0.96, 'rain': 6.11, 'uvi': 12.43}, {'dt': 1600401600, 'sunrise': 1600380661, 'sunset': 1600424649, 'temp': {'day': 28.84, 'min': 28.01, 'max': 28.84, 'night': 28.45, 'eve': 28.75, 'morn': 28.01}, 'feels_like': {'day': 28.65, 'night': 28.03, 'eve': 28.74, 'morn': 26.92}, 'pressure': 1008, 'humidity': 80, 'dew_point': 25.09, 'wind_speed': 9.47, 'wind_deg': 85, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10d'}], 'clouds': 100, 'pop': 1, 'rain': 38.2, 'uvi': 12.32}, {'dt': 1600488000, 'sunrise': 1600467077, 'sunset': 1600510989, 'temp': {'day': 28.54, 'min': 27.82, 'max': 28.81, 'night': 28.27, 'eve': 28.81, 'morn': 28.06}, 'feels_like': {'day': 28.54, 'night': 28.84, 'eve': 29.97, 'morn': 27.51}, 'pressure': 1012, 'humidity': 79, 'dew_point': 24.58, 'wind_speed': 8.77, 'wind_deg': 105, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10d'}], 'clouds': 100, 'pop': 0.99, 'rain': 28.17, 'uvi': 11.64}, {'dt': 1600574400, 'sunrise': 1600553493, 'sunset': 1600597328, 'temp': {'day': 28.81, 'min': 27.66, 'max': 29.08, 'night': 27.99, 'eve': 28.59, 'morn': 27.66}, 'feels_like': {'day': 29.81, 'night': 30.23, 'eve': 30.29, 'morn': 28.76}, 'pressure': 1013, 'humidity': 76, 'dew_point': 24.31, 'wind_speed': 7.01, 'wind_deg': 108, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 28, 'pop': 0.87, 'rain': 10.39, 'uvi': 12.62}, {'dt': 1600660800, 'sunrise': 1600639909, 'sunset': 1600683668, 'temp': {'day': 28.47, 'min': 27.03, 'max': 28.67, 'night': 27.03, 'eve': 28.13, 'morn': 27.51}, 'feels_like': {'day': 29.68, 'night': 28.52, 'eve': 29.34, 'morn': 30.06}, 'pressure': 1010, 'humidity': 76, 'dew_point': 23.91, 'wind_speed': 6.43, 'wind_deg': 126, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10d'}], 'clouds': 100, 'pop': 1, 'rain': 27.82, 'uvi': 11.78}]}```

<b>Ejemplo de JSON devuelto 2</b>:

```
{'lat': 1.3, 'lon': 103.83, 'timezone': 'Asia/Singapore', 'timezone_offset': 28800, 'current': {'dt': 1600080718, 'sunrise': 1600037804, 'sunset': 1600081440, 'temp': 26.83, 'feels_like': 31.42, 'pressure': 1008, 'humidity': 83, 'dew_point': 23.7, 'uvi': 13.55, 'clouds': 75, 'visibility': 10000, 'wind_speed': 1.5, 'wind_deg': 160, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'rain': {'1h': 1.89}}, 'minutely': [{'dt': 1600080720, 'precipitation': 1.8856}, {'dt': 1600080780, 'precipitation': 1.9404}, {'dt': 1600080840, 'precipitation': 1.9951999999999999}, {'dt': 1600080900, 'precipitation': 2.05}, {'dt': 1600080960, 'precipitation': 2.2716}, {'dt': 1600081020, 'precipitation': 2.4932}, {'dt': 1600081080, 'precipitation': 2.7148}, {'dt': 1600081140, 'precipitation': 2.9364}, {'dt': 1600081200, 'precipitation': 3.158}, {'dt': 1600081260, 'precipitation': 3.158}, {'dt': 1600081320, 'precipitation': 3.158}, {'dt': 1600081380, 'precipitation': 3.158}, {'dt': 1600081440, 'precipitation': 3.158}, {'dt': 1600081500, 'precipitation': 3.158}, {'dt': 1600081560, 'precipitation': 3.0732}, {'dt': 1600081620, 'precipitation': 2.9884}, {'dt': 1600081680, 'precipitation': 2.9036}, {'dt': 1600081740, 'precipitation': 2.8188}, {'dt': 1600081800, 'precipitation': 2.734}, {'dt': 1600081860, 'precipitation': 2.4948}, {'dt': 1600081920, 'precipitation': 2.2556000000000003}, {'dt': 1600081980, 'precipitation': 2.0164}, {'dt': 1600082040, 'precipitation': 1.7772000000000001}, {'dt': 1600082100, 'precipitation': 1.538}, {'dt': 1600082160, 'precipitation': 1.6404}, {'dt': 1600082220, 'precipitation': 1.7428}, {'dt': 1600082280, 'precipitation': 1.8452}, {'dt': 1600082340, 'precipitation': 1.9476}, {'dt': 1600082400, 'precipitation': 2.05}, {'dt': 1600082460, 'precipitation': 2.2716}, {'dt': 1600082520, 'precipitation': 2.4932}, {'dt': 1600082580, 'precipitation': 2.7148}, {'dt': 1600082640, 'precipitation': 2.9364}, {'dt': 1600082700, 'precipitation': 3.158}, {'dt': 1600082760, 'precipitation': 4.2558}, {'dt': 1600082820, 'precipitation': 5.3536}, {'dt': 1600082880, 'precipitation': 6.4514000000000005}, {'dt': 1600082940, 'precipitation': 7.549200000000001}, {'dt': 1600083000, 'precipitation': 8.647}, {'dt': 1600083060, 'precipitation': 8.2144}, {'dt': 1600083120, 'precipitation': 7.7818000000000005}, {'dt': 1600083180, 'precipitation': 7.3492}, {'dt': 1600083240, 'precipitation': 6.9166}, {'dt': 1600083300, 'precipitation': 6.484}, {'dt': 1600083360, 'precipitation': 5.8187999999999995}, {'dt': 1600083420, 'precipitation': 5.1536}, {'dt': 1600083480, 'precipitation': 4.4884}, {'dt': 1600083540, 'precipitation': 3.8232}, {'dt': 1600083600, 'precipitation': 3.158}, {'dt': 1600083660, 'precipitation': 3.0732}, {'dt': 1600083720, 'precipitation': 2.9884}, {'dt': 1600083780, 'precipitation': 2.9036}, {'dt': 1600083840, 'precipitation': 2.8188}, {'dt': 1600083900, 'precipitation': 2.734}, {'dt': 1600083960, 'precipitation': 2.5423999999999998}, {'dt': 1600084020, 'precipitation': 2.3508}, {'dt': 1600084080, 'precipitation': 2.1592000000000002}, {'dt': 1600084140, 'precipitation': 1.9676}, {'dt': 1600084200, 'precipitation': 1.776}, {'dt': 1600084260, 'precipitation': 1.776}, {'dt': 1600084320, 'precipitation': 1.776}], 'hourly': [{'dt': 1600077600, 'temp': 26.83, 'feels_like': 29.61, 'pressure': 1008, 'humidity': 83, 'dew_point': 23.7, 'clouds': 75, 'visibility': 10000, 'wind_speed': 4.09, 'wind_deg': 146, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.39, 'rain': {'1h': 0.1}}, {'dt': 1600081200, 'temp': 27.16, 'feels_like': 29.8, 'pressure': 1008, 'humidity': 81, 'dew_point': 23.61, 'clouds': 87, 'visibility': 10000, 'wind_speed': 4.21, 'wind_deg': 130, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'pop': 0.45, 'rain': {'1h': 3.16}}, {'dt': 1600084800, 'temp': 27.37, 'feels_like': 30.21, 'pressure': 1009, 'humidity': 78, 'dew_point': 23.19, 'clouds': 95, 'visibility': 10000, 'wind_speed': 3.58, 'wind_deg': 126, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10n'}], 'pop': 0.54, 'rain': {'1h': 1.54}}, {'dt': 1600088400, 'temp': 27.57, 'feels_like': 30.15, 'pressure': 1010, 'humidity': 77, 'dew_point': 23.17, 'clouds': 98, 'visibility': 10000, 'wind_speed': 3.94, 'wind_deg': 126, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.72, 'rain': {'1h': 0.32}}, {'dt': 1600092000, 'temp': 27.59, 'feels_like': 30.41, 'pressure': 1010, 'humidity': 76, 'dew_point': 22.98, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.44, 'wind_deg': 117, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.93, 'rain': {'1h': 0.83}}, {'dt': 1600095600, 'temp': 27.51, 'feels_like': 30.88, 'pressure': 1011, 'humidity': 76, 'dew_point': 23, 'clouds': 100, 'visibility': 10000, 'wind_speed': 2.59, 'wind_deg': 93, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10n'}], 'pop': 1, 'rain': {'1h': 1.23}}, {'dt': 1600099200, 'temp': 27.82, 'feels_like': 30.56, 'pressure': 1011, 'humidity': 74, 'dew_point': 22.87, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.38, 'wind_deg': 73, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 1, 'rain': {'1h': 0.41}}, {'dt': 1600102800, 'temp': 27.57, 'feels_like': 30.28, 'pressure': 1011, 'humidity': 75, 'dew_point': 22.74, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.4, 'wind_deg': 45, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 1, 'rain': {'1h': 0.23}}, {'dt': 1600106400, 'temp': 26.86, 'feels_like': 29.43, 'pressure': 1010, 'humidity': 78, 'dew_point': 22.82, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.58, 'wind_deg': 31, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 1, 'rain': {'1h': 0.71}}, {'dt': 1600110000, 'temp': 26.5, 'feels_like': 29.15, 'pressure': 1009, 'humidity': 80, 'dew_point': 22.86, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.52, 'wind_deg': 29, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.75, 'rain': {'1h': 0.17}}, {'dt': 1600113600, 'temp': 26.2, 'feels_like': 29.64, 'pressure': 1009, 'humidity': 81, 'dew_point': 22.7, 'clouds': 100, 'visibility': 10000, 'wind_speed': 2.32, 'wind_deg': 13, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'pop': 0.75, 'rain': {'1h': 0.43}}, {'dt': 1600117200, 'temp': 25.4, 'feels_like': 28, 'pressure': 1009, 'humidity': 87, 'dew_point': 23.24, 'clouds': 100, 'visibility': 3771, 'wind_speed': 3.84, 'wind_deg': 276, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10n'}], 'pop': 0.91, 'rain': {'1h': 3.67}}, {'dt': 1600120800, 'temp': 24.35, 'feels_like': 29.06, 'pressure': 1009, 'humidity': 93, 'dew_point': 23.19, 'clouds': 100, 'visibility': 5830, 'wind_speed': 0.88, 'wind_deg': 223, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10n'}], 'pop': 0.91, 'rain': {'1h': 6.56}}, {'dt': 1600124400, 'temp': 24.74, 'feels_like': 27.11, 'pressure': 1010, 'humidity': 87, 'dew_point': 22.46, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.66, 'wind_deg': 108, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'pop': 0.91, 'rain': {'1h': 2.17}}, {'dt': 1600128000, 'temp': 25.57, 'feels_like': 27.54, 'pressure': 1011, 'humidity': 78, 'dew_point': 21.66, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.48, 'wind_deg': 134, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.91, 'rain': {'1h': 0.13}}, {'dt': 1600131600, 'temp': 26.35, 'feels_like': 27.92, 'pressure': 1011, 'humidity': 77, 'dew_point': 22.02, 'clouds': 100, 'visibility': 10000, 'wind_speed': 4.46, 'wind_deg': 162, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.22}, {'dt': 1600135200, 'temp': 26.66, 'feels_like': 27.38, 'pressure': 1012, 'humidity': 77, 'dew_point': 22.33, 'clouds': 100, 'visibility': 10000, 'wind_speed': 5.9, 'wind_deg': 165, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.14}, {'dt': 1600138800, 'temp': 26.53, 'feels_like': 26.69, 'pressure': 1011, 'humidity': 79, 'dew_point': 22.72, 'clouds': 100, 'visibility': 10000, 'wind_speed': 6.93, 'wind_deg': 165, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.11}, {'dt': 1600142400, 'temp': 26.83, 'feels_like': 27.46, 'pressure': 1011, 'humidity': 76, 'dew_point': 22.37, 'clouds': 99, 'visibility': 10000, 'wind_speed': 5.99, 'wind_deg': 169, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.1}, {'dt': 1600146000, 'temp': 26.95, 'feels_like': 27.56, 'pressure': 1010, 'humidity': 75, 'dew_point': 22.23, 'clouds': 99, 'visibility': 10000, 'wind_speed': 5.95, 'wind_deg': 187, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.04}, {'dt': 1600149600, 'temp': 27.3, 'feels_like': 28.16, 'pressure': 1009, 'humidity': 73, 'dew_point': 22.17, 'clouds': 99, 'visibility': 10000, 'wind_speed': 5.51, 'wind_deg': 192, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0.04}, {'dt': 1600153200, 'temp': 27.47, 'feels_like': 28.58, 'pressure': 1008, 'humidity': 75, 'dew_point': 22.74, 'clouds': 100, 'visibility': 10000, 'wind_speed': 5.61, 'wind_deg': 164, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0}, {'dt': 1600156800, 'temp': 26.93, 'feels_like': 28.25, 'pressure': 1007, 'humidity': 79, 'dew_point': 23.06, 'clouds': 100, 'visibility': 10000, 'wind_speed': 5.58, 'wind_deg': 160, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0}, {'dt': 1600160400, 'temp': 26.81, 'feels_like': 28.71, 'pressure': 1007, 'humidity': 78, 'dew_point': 22.85, 'clouds': 100, 'visibility': 10000, 'wind_speed': 4.5, 'wind_deg': 177, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0}, {'dt': 1600164000, 'temp': 26.84, 'feels_like': 29.54, 'pressure': 1007, 'humidity': 76, 'dew_point': 22.37, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.04, 'wind_deg': 200, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0}, {'dt': 1600167600, 'temp': 26.88, 'feels_like': 29.44, 'pressure': 1008, 'humidity': 75, 'dew_point': 22.2, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.11, 'wind_deg': 222, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0}, {'dt': 1600171200, 'temp': 26.9, 'feels_like': 28.72, 'pressure': 1009, 'humidity': 76, 'dew_point': 22.53, 'clouds': 100, 'visibility': 10000, 'wind_speed': 4.34, 'wind_deg': 230, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600174800, 'temp': 26.77, 'feels_like': 28.73, 'pressure': 1010, 'humidity': 78, 'dew_point': 22.7, 'clouds': 100, 'visibility': 10000, 'wind_speed': 4.38, 'wind_deg': 230, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600178400, 'temp': 26.83, 'feels_like': 29.11, 'pressure': 1010, 'humidity': 77, 'dew_point': 22.49, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.8, 'wind_deg': 240, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600182000, 'temp': 26.87, 'feels_like': 29.5, 'pressure': 1011, 'humidity': 77, 'dew_point': 22.56, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.33, 'wind_deg': 233, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600185600, 'temp': 26.93, 'feels_like': 29.77, 'pressure': 1011, 'humidity': 76, 'dew_point': 22.43, 'clouds': 100, 'visibility': 10000, 'wind_speed': 2.91, 'wind_deg': 215, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600189200, 'temp': 26.8, 'feels_like': 29.8, 'pressure': 1010, 'humidity': 75, 'dew_point': 22.19, 'clouds': 100, 'visibility': 10000, 'wind_speed': 2.42, 'wind_deg': 204, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600192800, 'temp': 26.81, 'feels_like': 29.85, 'pressure': 1010, 'humidity': 75, 'dew_point': 22.04, 'clouds': 100, 'visibility': 10000, 'wind_speed': 2.37, 'wind_deg': 208, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600196400, 'temp': 26.78, 'feels_like': 29.5, 'pressure': 1009, 'humidity': 75, 'dew_point': 22, 'clouds': 99, 'visibility': 10000, 'wind_speed': 2.8, 'wind_deg': 212, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600200000, 'temp': 26.87, 'feels_like': 29.28, 'pressure': 1009, 'humidity': 74, 'dew_point': 22.05, 'clouds': 99, 'visibility': 10000, 'wind_speed': 3.15, 'wind_deg': 210, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600203600, 'temp': 26.92, 'feels_like': 29.19, 'pressure': 1009, 'humidity': 76, 'dew_point': 22.33, 'clouds': 100, 'visibility': 10000, 'wind_speed': 3.72, 'wind_deg': 211, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600207200, 'temp': 26.87, 'feels_like': 28.91, 'pressure': 1009, 'humidity': 77, 'dew_point': 22.56, 'clouds': 100, 'visibility': 10000, 'wind_speed': 4.18, 'wind_deg': 217, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'pop': 0}, {'dt': 1600210800, 'temp': 26.76, 'feels_like': 29.15, 'pressure': 1009, 'humidity': 76, 'dew_point': 22.38, 'clouds': 99, 'visibility': 10000, 'wind_speed': 3.43, 'wind_deg': 209, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0}, {'dt': 1600214400, 'temp': 26.96, 'feels_like': 29.62, 'pressure': 1010, 'humidity': 77, 'dew_point': 22.67, 'clouds': 94, 'visibility': 10000, 'wind_speed': 3.35, 'wind_deg': 205, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'pop': 0}, {'dt': 1600218000, 'temp': 27.43, 'feels_like': 30.46, 'pressure': 1011, 'humidity': 75, 'dew_point': 22.73, 'clouds': 32, 'visibility': 10000, 'wind_speed': 2.85, 'wind_deg': 196, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'pop': 0}, {'dt': 1600221600, 'temp': 28.12, 'feels_like': 31.15, 'pressure': 1011, 'humidity': 73, 'dew_point': 22.87, 'clouds': 18, 'visibility': 10000, 'wind_speed': 3.01, 'wind_deg': 186, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'pop': 0.01}, {'dt': 1600225200, 'temp': 28.99, 'feels_like': 31.46, 'pressure': 1011, 'humidity': 71, 'dew_point': 23.22, 'clouds': 16, 'visibility': 10000, 'wind_speed': 4.11, 'wind_deg': 181, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'pop': 0.01}, {'dt': 1600228800, 'temp': 29.31, 'feels_like': 31.59, 'pressure': 1011, 'humidity': 71, 'dew_point': 23.55, 'clouds': 15, 'visibility': 10000, 'wind_speed': 4.63, 'wind_deg': 185, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'pop': 0.07}, {'dt': 1600232400, 'temp': 29.38, 'feels_like': 31.89, 'pressure': 1010, 'humidity': 72, 'dew_point': 23.84, 'clouds': 12, 'visibility': 10000, 'wind_speed': 4.55, 'wind_deg': 193, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.39, 'rain': {'1h': 0.2}}, {'dt': 1600236000, 'temp': 29.33, 'feels_like': 31.71, 'pressure': 1009, 'humidity': 72, 'dew_point': 23.86, 'clouds': 12, 'visibility': 10000, 'wind_speed': 4.7, 'wind_deg': 199, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.27, 'rain': {'1h': 0.26}}, {'dt': 1600239600, 'temp': 29.61, 'feels_like': 31.63, 'pressure': 1008, 'humidity': 70, 'dew_point': 23.56, 'clouds': 44, 'visibility': 10000, 'wind_speed': 5.04, 'wind_deg': 194, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'pop': 0.33, 'rain': {'1h': 0.11}}, {'dt': 1600243200, 'temp': 29.83, 'feels_like': 31.8, 'pressure': 1007, 'humidity': 68, 'dew_point': 23.48, 'clouds': 66, 'visibility': 10000, 'wind_speed': 4.89, 'wind_deg': 186, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'pop': 0.2}, {'dt': 1600246800, 'temp': 29.84, 'feels_like': 31.89, 'pressure': 1007, 'humidity': 67, 'dew_point': 23.25, 'clouds': 71, 'visibility': 10000, 'wind_speed': 4.59, 'wind_deg': 171, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'pop': 0.21}], 'daily': [{'dt': 1600059600, 'sunrise': 1600037804, 'sunset': 1600081440, 'temp': {'day': 28.35, 'min': 26.02, 'max': 28.35, 'night': 27.36, 'eve': 27.17, 'morn': 26.31}, 'feels_like': {'day': 30.32, 'night': 30.77, 'eve': 30.14, 'morn': 30.57}, 'pressure': 1008, 'humidity': 75, 'dew_point': 23.68, 'wind_speed': 5.07, 'wind_deg': 186, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 99, 'pop': 1, 'rain': 4.65, 'uvi': 13.55}, {'dt': 1600146000, 'sunrise': 1600124184, 'sunset': 1600167816, 'temp': {'day': 27.3, 'min': 25.41, 'max': 27.3, 'night': 26.87, 'eve': 26.9, 'morn': 25.57}, 'feels_like': {'day': 28.16, 'night': 29.5, 'eve': 28.72, 'morn': 27.54}, 'pressure': 1009, 'humidity': 73, 'dew_point': 22.17, 'wind_speed': 5.51, 'wind_deg': 192, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10d'}], 'clouds': 99, 'pop': 1, 'rain': 14.49, 'uvi': 13.84}, {'dt': 1600228800, 'sunrise': 1600210564, 'sunset': 1600254191, 'temp': {'day': 28.99, 'min': 26.81, 'max': 29.84, 'night': 28.13, 'eve': 29.84, 'morn': 26.92}, 'feels_like': {'day': 31.46, 'night': 30.61, 'eve': 31.89, 'morn': 29.19}, 'pressure': 1011, 'humidity': 71, 'dew_point': 23.22, 'wind_speed': 4.11, 'wind_deg': 181, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': 16, 'pop': 0.56, 'rain': 1.41, 'uvi': 13.87}, {'dt': 1600315200, 'sunrise': 1600296944, 'sunset': 1600340567, 'temp': {'day': 29.29, 'min': 27.32, 'max': 29.29, 'night': 28.05, 'eve': 28.91, 'morn': 27.32}, 'feels_like': {'day': 32.22, 'night': 30.61, 'eve': 31.1, 'morn': 29.85}, 'pressure': 1010, 'humidity': 69, 'dew_point': 23.02, 'wind_speed': 3.3, 'wind_deg': 171, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': 92, 'pop': 0.7, 'rain': 2.62, 'uvi': 14.57}, {'dt': 1600401600, 'sunrise': 1600383324, 'sunset': 1600426943, 'temp': {'day': 29.12, 'min': 27.42, 'max': 29.12, 'night': 28.16, 'eve': 29.08, 'morn': 27.42}, 'feels_like': {'day': 30.59, 'night': 31.31, 'eve': 30.61, 'morn': 29.64}, 'pressure': 1009, 'humidity': 73, 'dew_point': 23.89, 'wind_speed': 6.02, 'wind_deg': 178, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 55, 'pop': 0.99, 'rain': 5.14, 'uvi': 13.91}, {'dt': 1600488000, 'sunrise': 1600469704, 'sunset': 1600513318, 'temp': {'day': 29.67, 'min': 27.59, 'max': 30.01, 'night': 28.05, 'eve': 29.93, 'morn': 27.59}, 'feels_like': {'day': 31.85, 'night': 30.73, 'eve': 31.06, 'morn': 29.46}, 'pressure': 1009, 'humidity': 72, 'dew_point': 24.17, 'wind_speed': 5.25, 'wind_deg': 172, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': 52, 'pop': 0.38, 'rain': 0.93, 'uvi': 14}, {'dt': 1600574400, 'sunrise': 1600556084, 'sunset': 1600599694, 'temp': {'day': 30.1, 'min': 27.81, 'max': 30.37, 'night': 28.54, 'eve': 29.69, 'morn': 27.81}, 'feels_like': {'day': 33.32, 'night': 32.33, 'eve': 32.04, 'morn': 30.82}, 'pressure': 1010, 'humidity': 70, 'dew_point': 24.14, 'wind_speed': 3.72, 'wind_deg': 116, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': 100, 'pop': 0.85, 'rain': 2.46, 'uvi': 14.34}, {'dt': 1600660800, 'sunrise': 1600642464, 'sunset': 1600686070, 'temp': {'day': 28.86, 'min': 28.07, 'max': 29.92, 'night': 28.25, 'eve': 29.92, 'morn': 28.13}, 'feels_like': {'day': 32.17, 'night': 31.25, 'eve': 32.99, 'morn': 31.87}, 'pressure': 1010, 'humidity': 72, 'dew_point': 23.51, 'wind_speed': 3, 'wind_deg': 137, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'clouds': 100, 'pop': 0.96, 'rain': 6.37, 'uvi': 13.86}]}
```